In [1]:
from dataclasses import dataclass
from string import ascii_lowercase

# https://en.wikipedia.org/wiki/Glossary_of_chess

FILES = "abcdefgh"
RANKS = list(range(1, 9))

KNIGHT_DELTAS = [
    (+2, +1),
    (+1, +2),
    (+2, -1),
    (-1, +2),
    (-2, +1),
    (+1, -2),
    (-2, -1),
    (-1, -2),
]

ORTHOGONAL_DELTAS = [
    (+0, +1),
    (+0, -1),
    (+1, +0),
    (-1, +0),
]

DIAGONAL_DELTAS = [
    (+1, +1),
    (+1, -1),
    (-1, +1),
    (-1, -1),
]


@dataclass(frozen=True)
class Position:
    x: int
    y: int

    @classmethod
    def from_square(cls, square: str):
        cls.validate_square(square)
        return cls(FILES.index(square[0]) + 1, int(square[1]))

    @staticmethod
    def validate_square(square):
        if len(square) != 2:
            raise ValueError("Square must have exactly two characters.")
        if not square[0] in FILES:
            raise ValueError("Letter must be lowercase between a and h.")
        if int(square[1]) < 1 or int(square[1]) > 8:
            raise ValueError("Integer must between 1 and 8.")

    def __eq__(self, other: object) -> bool:
        return self.x == other.x and self.y == other.y

    @property
    def square(self):
        return FILES[self.x - 1] + str(self.y)

    def __key__(self):
        return self.square

    def __repr__(self):
        return f"<Position {self.square}>"


class Mark:
    def __init__(self, p: Position, symbol: str = "*"):
        self.p = p
        self.symbol = symbol

    def __repr__(self) -> str:
        return f"<Mark {self.symbol}{self.p.square}>"


class Piece:
    def __init__(self, square: str, symbol="P"):
        self.p = Position.from_square(square)
        self.symbol = symbol

    def __repr__(self) -> str:
        return f"<Piece {self.symbol}{self.p.square}>"

    def create_lookup(self, board):
        queue = [self]
        d = {self.p: None}

        while queue:
            node = queue.pop(0)
            for p in node.neighbours(board):
                if p not in d:
                    queue.append(type(node)(p.square))
                    d[p] = node.p
        return d

    def shortest_path(self, board, square: str):
        d = self.create_lookup(board)
        node = Position.from_square(square)
        path = []
        while node is not None and node != self.p:
            # target is not reachable
            if node not in d:
                raise Exception(f"{self} cannot reach {square}.")
            path.append(node)
            node = d[node]
        return path[::-1]


class Board:
    def __init__(self, n: int = 8, pieces=None, marks=None):
        if n < 1 or n > 8:
            raise ValueError(f"Board size must be between 1 and 8.")
        self.pieces = pieces or []
        self.marks = marks or []
        self.n = n

    def add_waypoints(self, piece, target):
        marks = []
        for i, p in enumerate(piece.shortest_path(self, target)):
            marks.append(Mark(p, symbol=str(i+1)))
        return self.add_marks(*marks)

    def add_marks(self, *marks):
        return Board(
            n=self.n,
            pieces=self.pieces,
            marks=[*self.marks, *marks],
        )

    def add_neighbours(self, piece, degree=1):
        queue = [(0, piece)]
        positions = set()
        marks = []
        while queue:
            i, node = queue.pop(0)
            i += 1
            if i > degree:
                break
            for p in node.neighbours(self):
                if p not in positions:
                    new_piece = type(node)(p.square)
                    queue.append((i, new_piece))
                    marks.append(Mark(p, symbol=i))
                    positions.add(p)
        return self.add_marks(*marks)

    def add_pieces(self, *pieces):
        for piece in pieces:
            if not self.is_accessible(piece.p):
                raise ValueError(f"Position {piece.p} is not accessible.")
        return Board(
            n=self.n,
            pieces=[*self.pieces, *pieces],
            marks=self.marks,
        )

    def is_contained(self, p: Position):
        return p.x >= 1 and p.x <= self.n and p.y >= 1 and p.y <= self.n

    def is_accessible(self, p: Position):
        return self.is_contained(p) and not self.is_occupied(p)

    def is_occupied(self, p: Position):
        for piece in self.pieces:
            if piece.p == p:
                return True
        return False

    def __str__(self):
        """
        Prints the board with pieces and marks.
        Assumes that pieces do not overlap.
        If marks overlap, the first is shown.
        """
        board = []
        board.append(" ".join(" " + FILES[: self.n]))
        for row in range(self.n, 0, -1):
            line = f"{row}"
            for col in range(1, self.n + 1):
                for item in [*self.pieces, *self.marks]:
                    if item.p == Position(col, row):
                        line += f" {item.symbol}"
                        break
                else:
                    line += " ."
            board.append(line)
        return "\n".join(board)


class Knight(Piece):
    def __init__(self, square: str, symbol="K"):
        super().__init__(square, symbol)

    def neighbours(self, board: Board):
        positions = []
        for dx, dy in KNIGHT_DELTAS:
            p = Position(self.p.x + dx, self.p.y + dy)
            if board.is_accessible(p):
                positions.append(p)
        return positions


class Bishop(Piece):
    def __init__(self, square: str, symbol="B"):
        super().__init__(square, symbol)

    def neighbours(self, board: Board):
        positions = []
        for dx, dy in DIAGONAL_DELTAS:
            p = Position(self.p.x, self.p.y)
            while board.is_accessible(p := Position(p.x + dx, p.y + dy)):
                positions.append(p)
        return positions


class Tower(Piece):
    def __init__(self, square: str, symbol="T"):
        super().__init__(square, symbol)

    def neighbours(self, board: Board):
        positions = []
        for dx, dy in ORTHOGONAL_DELTAS:
            p = Position(self.p.x, self.p.y)
            while board.is_accessible(p := Position(p.x + dx, p.y + dy)):
                positions.append(p)
        return positions

In [8]:
k = Knight("d5")
b = Bishop("b3")
t = Tower("f2")

board = (
    Board()
    #.add_pieces(k, b, t)
    #.add_neighbours(t, degree=1)
    #.add_waypoints(t, "h2")
)

print(board)

  a b c d e f g h
8 . . . . . . . .
7 . . . . . . . .
6 . . . . . . . .
5 . . . . . . . .
4 . . . . . . . .
3 . . . . . . . .
2 . . . . . . . .
1 . . . . . . . .
